In [2]:
import tables

In [3]:
import processing_pipeline as pp

path = "Dataset/open-images-v7/train"

current_process = pp.preprocess()
current_process.set_dataframe(path)
current_process.load_data_img(path)
current_process.process_one()
df = current_process.augmentation_process()
#current_process.out_put("/")

INFO:root:Loading labels started...
INFO:root:Dataset labels finished
INFO:root:Loading Images from Dataset/open-images-v7/train started...
INFO:root:1440 Images loaded
INFO:root:Preprocess one started...
INFO:root:Preprocess one finish
INFO:root:Data Augmentation started...
INFO:root:Data Augmentation finished


In [4]:
# Get dataset and label hotdogs
df = current_process.rebuild_dataset()
df['n_label'] = df.label.apply(lambda x: 1 if x=="Hot dog" else 0)

In [5]:
def sample(df, fraction=0.5):
  # Assuming you have a df
  # Randomly sample fraction% of the DataFrame rows without replacement
  sampled_df = df.sample(frac=fraction, replace=False)
  return sampled_df

In [6]:
def hog_features(X_train, X_test):
    import numpy as np
    from skimage.feature import hog

    # Assume you have X train, y train with image
    # Extract HOG features for each image
    hog_features_train = []
    hog_features_test = []

    for image in X_train:
        hog_features = hog(image, orientations=9, pixels_per_cell=(16, 16), cells_per_block=(2, 2), visualize=False)
        hog_features_train.append(hog_features)

    for image in X_test:
        hog_features = hog(image, orientations=9, pixels_per_cell=(16, 16), cells_per_block=(2, 2), visualize=False)
        hog_features_test.append(hog_features)

    # Convert the features lists to numpy arrays
    X_train_hog = np.array(hog_features_train)
    X_test_hog = np.array(hog_features_test)
    
    return X_train_hog, X_test_hog

In [7]:
def get_raw_pixel_values(X_train, X_test):
  # Raw pixel values
  # You can flatten the image arrays and use the raw pixel values as features. 
  # Each pixel value will be considered as a separate feature. 
  # This approach captures the raw intensity values of each pixel in the image.
  # Assuming your image data is stored in a DataFrame called 'df' with a column named 'image'
  # Convert the image arrays to a numpy array
  # Reshape the images to a 2D array (num_samples, num_pixels)
  num_samples = X_train.shape[0]
  raw_values_train = images.reshape(num_samples, -1)
  
  num_samples = X_test.shape[0]
  raw_values_test = images.reshape(num_samples, -1)
  
  return raw_values_train, raw_values_test

def get_histograms(X_train, X_test):
  import numpy as np
  #Histogram of Pixel Intensities:
  #You can calculate histograms of pixel intensities to capture the distribution of intensity values across the image. 
  # This approach provides information about the overall brightness and contrast of the image.
  # Calculate histograms for each image
  histograms = np.array([np.histogram(image, bins=256, range=(0, 256))[0] for image in X_train])
  # Normalize the histograms
  histogram_train = histograms / histograms.sum(axis=1, keepdims=True)
  
  histograms = np.array([np.histogram(image, bins=256, range=(0, 256))[0] for image in X_test])
  # Normalize the histograms
  histogram_test = histograms / histograms.sum(axis=1, keepdims=True)
  
  return histogram_train, histogram_test

def get_canny_edges(X_train, X_test):
  import cv2
  import numpy as np
  import pandas as pd
  # Apply Canny edge detection to each image
  edges = np.array([cv2.Canny((image*255).astype(np.uint8), threshold1=100, threshold2=200) for image in X_train])
  # Reshape the edge arrays to a 2D array (num_samples, num_pixels)
  num_samples = X_train.shape[0]
  canny_edge_train = edges.reshape(num_samples, -1)
  
  edges = np.array([cv2.Canny((image*255).astype(np.uint8), threshold1=100, threshold2=200) for image in X_test])
  num_samples = X_test.shape[0]
  canny_edge_test = edges.reshape(num_samples, -1)
  
  return canny_edge_train, canny_edge_test

In [8]:
def preprocess_images(X_train, X_test):
  import numpy as np
  from skimage.feature import hog
  from sklearn.preprocessing import StandardScaler
  import pandas as pd

  print('Starting preprocess...')
  print('Getting HOG features...')
  X_train_hog, X_test_hog = hog_features(X_train, X_test)
  print('Getting Histograms...')
  X_train_histogram, X_test_histogram = get_histograms(X_train, X_test)
  
  X_train_expanded = pd.concat([pd.DataFrame(X_train_hog), pd.DataFrame(X_train_histogram)], axis=1)
  X_test_expanded = pd.concat([pd.DataFrame(X_test_hog), pd.DataFrame(X_test_histogram)], axis=1)
  
  print('Merging & scaling data...')
  # Scale the features
  scaler = StandardScaler()
  X_train_scaled = scaler.fit_transform(X_train_expanded)
  X_test_scaled = scaler.transform(X_test_expanded)
  
  return X_train_scaled, X_test_scaled

In [9]:
def reduce_dimension_pca(X_train, X_test, n_components):
    import pandas as pd
    from sklearn.decomposition import PCA

    print("Performing PCA...")
    # Perform PCA on training data
    pca = PCA(n_components=n_components)
    X_train_pca = pca.fit_transform(X_train)
    
    # Calculate percentage of variation explained
    explained_variance_ratio = pca.explained_variance_ratio_
    total_variance_explained = sum(explained_variance_ratio[:n_components]) * 100
    print("Total variance explaned: {}".format(total_variance_explained))
    
    print("Applying PCA...")
    # Apply PCA transformation to test data
    X_test_pca = pca.transform(X_test)
    
    # Create data frames from reduced dimension data
    X_train_pca_df = pd.DataFrame(X_train_pca, columns=[f'PC{i+1}' for i in range(n_components)])
    X_test_pca_df = pd.DataFrame(X_test_pca, columns=[f'PC{i+1}' for i in range(n_components)])
    
    return X_train_pca_df, X_test_pca_df


In [10]:
def get_predictions(X, y, model):
    import numpy as np
    from skimage.feature import hog
    from sklearn.model_selection import train_test_split
    from sklearn.preprocessing import StandardScaler
    from sklearn.svm import SVC
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.metrics import accuracy_score
    import pandas as pd
    import time

    # Assuming your image data is stored in a DataFrame called 'df' with a column named 'image'
    # Convert the image arrays to a numpy array
    images = np.array(X.tolist())
    labels = np.array(y)

    # Split the data into training and testing sets
    print('Spliting train/test...')
    X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)
    
    # Apply preprocess
    X_train_scaled, X_test_scaled = preprocess_images(X_train, X_test)
    
    # Reduce dimensionality
    n_components = 10
    X_train_pca_df, X_test_pca_df = reduce_dimension_pca(X_train_scaled, X_test_scaled, n_components)

    print('Training model...')
    # Train the model
    classifier = model
    classifier.fit(X_train_pca_df, y_train)
    y_pred_train = classifier.predict(X_train_pca_df)

    print('Making predictions...')
    # Make predictions on the test set
    y_pred = classifier.predict(X_test_pca_df)

    # Calculate accuracy
    accuracy_train = accuracy_score(y_train, y_pred_train)
    accuracy_test = accuracy_score(y_test, y_pred)
    
    print("Train Accuracy: {}".format(accuracy_train))
    print("Test Accuracy: {}".format(accuracy_test))
    
    return y_pred_train, y_pred, accuracy_train, accuracy_test


In [14]:
# Test with all preprocess stages
# Get X, y
sampled_df = sample(df, 1)

X = sampled_df.img
y = sampled_df.n_label

import time

start = time.time()

# Create model
# Stacking 
from mlxtend.classifier import StackingClassifier
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression, Ridge
from sklearn.metrics import f1_score, accuracy_score,classification_report,confusion_matrix
from sklearn import metrics

# Define regularization parameters
regularization_params = {
    'logisticregression__C': 50,  # Specify the regularization strength for LogisticRegression
    'ridge__alpha': 0.5  # Specify the regularization strength for Ridge
}

m = StackingClassifier(
    classifiers=[
        LogisticRegression(C=regularization_params['logisticregression__C']),
        KNeighborsRegressor(n_neighbors=2),
        DecisionTreeClassifier(random_state=0)
    ],
    use_probas=False,
    meta_classifier=LogisticRegression(C=regularization_params['logisticregression__C'])
)

y_pred_train, y_pred, accuracy_train, accuracy_test = get_predictions(X, y, m)

end = time.time()

print("Total running time in minutes: {}".format((time.time() - start)/60))



Spliting train/test...
Starting preprocess...
Getting HOG features...
Getting Histograms...
Merging & scaling data...
Performing PCA...
Total variance explaned: 12.316759174607089
Applying PCA...
Training model...
Making predictions...
Train Accuracy: 1.0
Test Accuracy: 0.5986238532110092
Total running time in minutes: 4.230105817317963


In [18]:
# Test with all preprocess stages
# Get X, y
sampled_df = sample(df, 1)

X = sampled_df.img
y = sampled_df.n_label

import time

start = time.time()

# Create model
# Stacking 
from mlxtend.classifier import StackingClassifier
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, accuracy_score,classification_report,confusion_matrix
import xgboost as xgb
from sklearn import metrics

m = xgb.XGBClassifier()

y_pred_train, y_pred, accuracy_train, accuracy_test = get_predictions(X, y, m)

end = time.time()

print("Total running time in minutes: {}".format((time.time() - start)/60))



Spliting train/test...
Starting preprocess...
Getting HOG features...
Getting Histograms...
Getting Canny Edges...
Merging & scaling data...
Performing PCA...
Total variance explaned: 100.00000000000009
Applying PCA...
Training model...
Making predictions...
Train Accuracy: 0.710919540229885
Test Accuracy: 0.5229357798165137
Total running time in minutes: 4.705825118223826


In [13]:
# Test with just HOG Process
# Get X, y
sampled_df = sample(df, 0.5)

X = sampled_df.img
y = sampled_df.n_label

import time

start = time.time()

# Create model
# Stacking 
from mlxtend.classifier import StackingClassifier
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, accuracy_score,classification_report,confusion_matrix
from sklearn import metrics

m = StackingClassifier(
    classifiers=[
        LogisticRegression(),
        KNeighborsRegressor(n_neighbors=2),
        DecisionTreeClassifier(random_state=0)
    ],
    use_probas=False,
    meta_classifier=LogisticRegression()
)

y_pred_train, y_pred, accuracy_train, accuracy_test = get_predictions(X, y, m)

end = time.time()

print("Total running time in minutes: {}".format((time.time() - start)/60))
print("Train Accuracy: {}".format(accuracy_train))
print("Train Accuracy: {}".format(accuracy_test))


Spliting train/test...
Starting preprocess...
Getting HOG features...
Merging & scaling data...
Training model...
Making predictions...
Total running time in minutes: 2.1730300148328143
Train Accuracy: 1.0
Train Accuracy: 0.5688073394495413


In [15]:
# Test with just Canny Edges
# Get X, y
sampled_df = sample(df, 0.5)

X = sampled_df.img
y = sampled_df.n_label

import time

start = time.time()

# Create model
# Stacking 
from mlxtend.classifier import StackingClassifier
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, accuracy_score,classification_report,confusion_matrix
from sklearn import metrics

m = StackingClassifier(
    classifiers=[
        LogisticRegression(),
        KNeighborsRegressor(n_neighbors=2),
        DecisionTreeClassifier(random_state=0)
    ],
    use_probas=False,
    meta_classifier=LogisticRegression()
)

y_pred_train, y_pred, accuracy_train, accuracy_test = get_predictions(X, y, m)

end = time.time()

print("Total running time in minutes: {}".format((time.time() - start)/60))
print("Train Accuracy: {}".format(accuracy_train))
print("Train Accuracy: {}".format(accuracy_test))


Spliting train/test...
Starting preprocess...
Getting Canny Edges...
Merging & scaling data...
Training model...
Making predictions...
Total running time in minutes: 2.5255083163579304
Train Accuracy: 1.0
Train Accuracy: 0.4954128440366973


In [18]:
# Test with just Histograms
# Get X, y
sampled_df = sample(df, 0.5)

X = sampled_df.img
y = sampled_df.n_label

import time

start = time.time()

# Create model
# Stacking 
from mlxtend.classifier import StackingClassifier
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, accuracy_score,classification_report,confusion_matrix
from sklearn import metrics

m = StackingClassifier(
    classifiers=[
        LogisticRegression(),
        KNeighborsRegressor(n_neighbors=2),
        DecisionTreeClassifier(random_state=0)
    ],
    use_probas=False,
    meta_classifier=LogisticRegression()
)

y_pred_train, y_pred, accuracy_train, accuracy_test = get_predictions(X, y, m)

end = time.time()

print("Total running time in minutes: {}".format((time.time() - start)/60))
print("Train Accuracy: {}".format(accuracy_train))
print("Train Accuracy: {}".format(accuracy_test))


Spliting train/test...
Starting preprocess...
Getting Histograms...
Merging & scaling data...
Training model...
Making predictions...
Total running time in minutes: 0.8873202999432882
Train Accuracy: 0.7241379310344828
Train Accuracy: 0.591743119266055


In [9]:
# CNN

import torch
import numpy as np

# Convert each element in the Pandas Series to a NumPy array
arrays = [np.array(x) for x in X_train]
# Stack the NumPy arrays together to create a NumPy array with consistent shapes
array = np.stack(arrays)
# Convert the NumPy array to a PyTorch tensor
tensor_X_train = torch.from_numpy(array)

# Convert each element in the Pandas Series to a NumPy array
arrays = [np.array(x) for x in y_train]
# Stack the NumPy arrays together to create a NumPy array with consistent shapes
array = np.stack(arrays)
# Convert the NumPy array to a PyTorch tensor
tensor_y_train = torch.from_numpy(array)

# Convert each element in the Pandas Series to a NumPy array
arrays = [np.array(x) for x in X_test]
# Stack the NumPy arrays together to create a NumPy array with consistent shapes
array = np.stack(arrays)
# Convert the NumPy array to a PyTorch tensor
tensor_X_test = torch.from_numpy(array)

# Convert each element in the Pandas Series to a NumPy array
arrays = [np.array(x) for x in y_test]
# Stack the NumPy arrays together to create a NumPy array with consistent shapes
array = np.stack(arrays)
# Convert the NumPy array to a PyTorch tensor
tensor_y_test = torch.from_numpy(array)



In [17]:
import torch
import torch.nn as nn
import torch.optim as optim

img_size = 600

# Define the CNN model architecture
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)  # Update input channels to 1
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(64 * (img_size // 4) * (img_size // 4), 128)
        self.fc2 = nn.Linear(128, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.conv2(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.sigmoid(x)
        return x

# Create an instance of the CNN model
model = CNN()

# Define loss function and optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Convert your dataset to PyTorch tensors and dataloaders
# Assuming you have preprocessed tensors X_train, y_train, X_test, y_test

train_dataset = torch.utils.data.TensorDataset(tensor_X_train.float(), tensor_y_train)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)

test_dataset = torch.utils.data.TensorDataset(tensor_X_test.float(), tensor_y_test)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False)

# Train the model
for epoch in range(10):
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs = inputs.unsqueeze(1)  # Add extra dimension for grayscale channel
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels.unsqueeze(1).float())
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch + 1} - Loss: {running_loss / len(train_loader)}")

# Evaluate the model
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs = inputs.unsqueeze(1)  # Add extra dimension for grayscale channel
        outputs = model(inputs)
        predicted = (outputs >= 0.5).squeeze().long()
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
accuracy = correct / total
print(f"Test Accuracy: {accuracy}")


Epoch 1 - Loss: 49.20992986505682
Epoch 2 - Loss: 49.84848487160423
Epoch 3 - Loss: 50.132575711337005


: 

: 